In [1]:
#  IMPORTING LIBRARIES
import datetime as dt
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [2]:
from numpy import log as ln
import pandas_datareader as web
import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


In [3]:
def draw_graph(dataframe, columns, colors, title):
    """
    Draws a pyplot graph with up to 3 lines depending on the input.
    """
        
    layout = go.Layout(
        yaxis2 = dict(
            title="Normalized value", 
            side="right",
            tickmode = "auto"
        ),
        title = title)
    fig = go.Figure(layout = layout)
    
    for column, color in zip(columns, colors):
        fig.add_trace(
            add_line(dataframe.index, dataframe[column], color, column)
        )
    
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=7,
                        label="7d",
                        step="day",
                        stepmode="backward"),
                    dict(count=1,
                         label="1m",
                         step="month",
                         stepmode="backward"),
                    dict(count=6,
                         label="6m",
                         step="month",
                         stepmode="backward"),
                    dict(count=1,
                         label="YTD",
                         step="year",
                         stepmode="todate"),
                    dict(count=1,
                         label="1y",
                         step="year",
                         stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(
                visible=True
            ),
            type="date"
        )
    )
    fig.show()
    
def add_line(x_data,y_data, color, title):
    """Adds a pyplot line to a predefine pyplot figure."""
    return go.Scatter(
        mode="lines",
        x=x_data,
        y=y_data,
        line = dict(width=1),
        marker = dict(color=color),
        yaxis = "y1", name = title
    )

## Import Data

In [4]:
#  Example import, needs to be updated accordingly...
df = pd.read_csv("LSTM_results.csv")

In [5]:
df

,DateTime,Forecast,Actual
0,2018-05-22,0.330828,0.296319
1,2018-05-23,0.333239,0.299195
2,2018-05-24,0.328026,0.282542
3,2018-05-25,0.322993,0.291667
4,2018-05-27,0.310946,0.277665
5,2018-05-28,0.305424,0.257754
6,2018-05-29,0.296629,0.286318
7,2018-05-30,0.288409,0.292521
8,2018-05-31,0.293996,0.284924
9,2018-06-01,0.302381,0.286902


In [6]:
def calc_momentum(_df, _momentum):
    _df['F_momentum'] = _df['Forecast'] - _df['Forecast'].shift(_momentum)
    _df['A_momentum'] = _df['Actual'] - _df['Actual'].shift(_momentum)
    return _df

In [7]:
df_test = calc_momentum(df, 2)

In [8]:
df_test

,DateTime,Forecast,Actual,F_momentum,A_momentum
0,2018-05-22,0.330828,0.296319,NaN,NaN
1,2018-05-23,0.333239,0.299195,NaN,NaN
2,2018-05-24,0.328026,0.282542,-0.002801,-0.013777
3,2018-05-25,0.322993,0.291667,-0.010246,-0.007529
4,2018-05-27,0.310946,0.277665,-0.017081,-0.004877
5,2018-05-28,0.305424,0.257754,-0.017570,-0.033913
6,2018-05-29,0.296629,0.286318,-0.014316,0.008652
7,2018-05-30,0.288409,0.292521,-0.017015,0.034767
8,2018-05-31,0.293996,0.284924,-0.002634,-0.001393
9,2018-06-01,0.302381,0.286902,0.013973,-0.005618


In [13]:
df_test_2 = df_test.set_index("DateTime")

In [14]:
draw_graph(df_test_2, ["F_momentum", "A_momentum"], ["#eb1717","#0059ff"], "Momentum")